In [3]:
import pandas as pd
import zipfile
import os

zip_path = "Electric_Vehicle_Population_Data.zip"
extract_dir = "EV_Dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

for file in os.listdir(extract_dir):
    if file.endswith(".csv"):
        dataset_path = os.path.join(extract_dir, file)
        print("Dataset found:", dataset_path)

df = pd.read_csv(dataset_path)

print("Shape of dataset:", df.shape)
df.head()


Dataset found: EV_Dataset/Electric_Vehicle_Population_Data.csv
Shape of dataset: (264628, 17)


,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,WA1E2AFY8R,Thurston,Olympia,WA,98512.0,2024,AUDI,Q5 E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,23.0,0.0,22.0,263239938,POINT (-122.90787 46.9461),PUGET SOUND ENERGY INC,5.306701e+10
1,WAUUPBFF4J,Yakima,Wapato,WA,98951.0,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16.0,0.0,15.0,318160860,POINT (-120.42083 46.44779),PACIFICORP,5.307794e+10
2,1N4AZ0CP0F,King,Seattle,WA,98125.0,2015,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,184963586,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,WA1VAAGE5K,King,Kent,WA,98031.0,2019,AUDI,E-TRON,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,204.0,0.0,11.0,259426821,POINT (-122.17743 47.41185),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,7SAXCAE57N,Snohomish,Bothell,WA,98021.0,2022,TESLA,MODEL X,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,1.0,208182236,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10


In [4]:
df.info()

df.describe()

# Check missing values
df.isnull().sum()

# Unique values in target column
df['Electric Vehicle Type'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264628 entries, 0 to 264627
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         264628 non-null  object 
 1   County                                             264619 non-null  object 
 2   City                                               264619 non-null  object 
 3   State                                              264628 non-null  object 
 4   Postal Code                                        264619 non-null  float64
 5   Model Year                                         264628 non-null  int64  
 6   Make                                               264628 non-null  object 
 7   Model                                              264628 non-null  object 
 8   Electric Vehicle Type                              264628 non-null  object

,count
Electric Vehicle Type,
Battery Electric Vehicle (BEV),210575
Plug-in Hybrid Electric Vehicle (PHEV),54053


In [5]:
# Drop unnecessary columns
df = df.drop([
    'VIN (1-10)',
    'DOL Vehicle ID',
    'Vehicle Location'
], axis=1)

In [6]:

num_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in num_cols:
    df[col].fillna(df[col].median(), inplace=True)

cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)


/tmp/ipython-input-1005579922.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-1005579922.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

In [8]:
# Check again for missing values
print(df.isnull().sum().sum(), "missing values left")

# Display cleaned dataset info
df.info()

# Preview cleaned data
df.head()


0 missing values left
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264628 entries, 0 to 264627
Data columns (total 14 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   County                                             264628 non-null  int64  
 1   City                                               264628 non-null  int64  
 2   State                                              264628 non-null  int64  
 3   Postal Code                                        264628 non-null  float64
 4   Model Year                                         264628 non-null  int64  
 5   Make                                               264628 non-null  int64  
 6   Model                                              264628 non-null  int64  
 7   Electric Vehicle Type                              264628 non-null  int64  
 8   Clean Alternative Fuel Vehicle (CAFV) Eligibility  2

,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,Electric Utility,2020 Census Tract
0,219,556,48,98512.0,2024,2,126,1,2,23.0,0.0,22.0,73,5.306701e+10
1,236,815,48,98951.0,2018,2,10,1,2,16.0,0.0,15.0,65,5.307794e+10
2,106,690,48,98125.0,2015,31,98,0,0,84.0,0.0,46.0,57,5.303300e+10
3,106,367,48,98031.0,2019,2,44,0,0,204.0,0.0,11.0,74,5.303303e+10
4,202,67,48,98021.0,2022,39,105,0,1,0.0,0.0,1.0,73,5.306105e+10


In [9]:
# Define target and features
X = df.drop('Electric Vehicle Type', axis=1)
y = df['Electric Vehicle Type']

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: (264628, 13)
Target shape: (264628,)


In [10]:
# Step 4.1: Split dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Training set:", X_train.shape)
print("Testing set:", X_test.shape)


Training set: (211702, 13)
Testing set: (52926, 13)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

# Predict on test data
y_pred = log_reg.predict(X_test)

# Evaluate model
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Logistic Regression Accuracy: 0.7957336658731059

Classification Report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89     42115
           1       0.00      0.00      0.00     10811

    accuracy                           0.80     52926
   macro avg       0.40      0.50      0.44     52926
weighted avg       0.63      0.80      0.71     52926


Confusion Matrix:
 [[42115     0]
 [10811     0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
import joblib

joblib.dump(log_reg, "ev_type_predictor_model.pkl")
joblib.dump(list(X_train.columns), "feature_columns.pkl")

['feature_columns.pkl']

In [13]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.8 MB/s eta 0:00:00


In [14]:
from pyngrok import ngrok
ngrok.set_auth_token("34phGAylXhX5TioiJC6NTQSMshz_2gFVDNVtXdYQaV22eHwXj")

In [15]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import os

st.set_page_config(page_title="EV Type Predictor", layout="wide")
st.title("Electric Vehicle Type Predictor")

MODEL_PATH = "ev_type_predictor_model.pkl"
FEATURES_PATH = "feature_columns.pkl"

if not os.path.exists(MODEL_PATH):
    st.error("❌ Model file not found. Upload ev_type_predictor_model.pkl.")
elif not os.path.exists(FEATURES_PATH):
    st.error("❌ Feature file not found. Upload feature_columns.pkl.")
else:
    model = joblib.load(MODEL_PATH)
    feature_cols = joblib.load(FEATURES_PATH)
    st.success("✅ Model and feature columns loaded successfully!")

    uploaded_file = st.file_uploader("📤 Upload your Electric Vehicle dataset (CSV)", type=["csv"])

    if uploaded_file is not None:
        df = pd.read_csv(uploaded_file)
        st.subheader("📊 Uploaded Dataset Preview")
        st.write(df.head())

        st.subheader("🔍 Select a row for prediction")
        index = st.number_input("Enter row index:", min_value=0, max_value=max(0, len(df)-1), value=0, step=1)
        input_data = df.iloc[[index]].copy()
        st.write(input_data)

        # Drop unnecessary columns if present
        drop_cols = ['VIN (1-10)', 'DOL Vehicle ID', 'Vehicle Location']
        for col in drop_cols:
            if col in input_data.columns:
                input_data.drop(col, axis=1, inplace=True)

        # Apply same encoding as training
        input_data = pd.get_dummies(input_data)

        # Align columns with training data
        input_data = input_data.reindex(columns=feature_cols, fill_value=0)

        try:
            prediction = model.predict(input_data)
            st.subheader("🔮 Prediction Result")
            type_map = {0: "Battery Electric Vehicle (BEV)", 1: "Plug-in Hybrid Electric Vehicle (PHEV)"}
            predicted_type = type_map.get(prediction[0], "Unknown Type")
            st.subheader("🔮 Prediction Result")
            st.success(f"The predicted EV Type is: **{predicted_type}**")
        except Exception as e:
            st.error("⚠️ Prediction failed.")
            st.write(e)


Writing app.py


In [16]:
# start streamlit in background
import os, time, signal
# kill any process using port 8501 first (safe)
!npx kill-port 8501 || true
get_ipython().system_raw("streamlit run app.py --server.port 8501 &> /content/streamlit.log &")
time.sleep(2)
print("Streamlit started (logs -> /content/streamlit.log).")


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏Need to install the following packages:
kill-port@2.0.1
Ok to proceed? (y) y

⠙⠹⠸⠼⠴Could not kill process on port 8501. No process running on port.
⠙Streamlit started (logs -> /content/streamlit.log).


In [17]:
!pkill ngrok


In [19]:
from pyngrok import ngrok

# Optional: reset your token if needed
ngrok.set_auth_token("34phGAylXhX5TioiJC6NTQSMshz_2gFVDNVtXdYQaV22eHwXj")

# Start a new tunnel on port 8501
public_url = ngrok.connect(8501)
print("Streamlit App URL:", public_url)


Streamlit App URL: NgrokTunnel: "https://klara-homosporous-heavenly.ngrok-free.dev" -> "http://localhost:8501"


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))


Accuracy: 0.9998299512526925
